In [1]:
import tensorflow as tf
import os
import random
import numpy as np
import matplotlib.pyplot as plt
import pickle
import time
print(tf.version.VERSION)

2.1.0


In [2]:
DATADIR0 = r"D:\K14\Dataset\K14-Dataset-1.pickle"
DATADIR1 = r"D:\K14\Dataset\K14-Dataset-0-Stuff.pickle"
DATADIR2 = os.path.dirname(r"C:\Users\8holz\Dokumente\GitHub\K14\Training_Protocol\\")
DATADIR3 = None

checkpoint_path = "C:/Users/8holz/Dokumente/GitHub/K14/Training_Protocol/Model-Color-UNET-Terrosristhunt/weights/saved_model.pb"
checkpoint_dir = os.path.dirname(checkpoint_path)

In [3]:
IMG_WIDTH = 512
IMG_HEIGHT = 512
IMG_CHANNELS = 3

In [4]:
Data = pickle.load(open(DATADIR0, "rb"))
X_train = Data["TrainImages0"]
Y_train = Data["TrainImages1"]

X_test = Data["TestImages0"]
Y_test = Data["TestImages1"]

In [5]:
# Define a simple sequential model
def create_model(IMG_HEIGHT, IMG_WIDTH, IMG_CHANNELS):
    #Build the model
    inputs = tf.keras.layers.Input((IMG_HEIGHT, IMG_WIDTH, IMG_CHANNELS))
    s = tf.keras.layers.Lambda(lambda x: x / 255)(inputs)

    #Contraction path
    c1 = tf.keras.layers.Conv2D(16, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(s)
    c1 = tf.keras.layers.Dropout(0.1)(c1)
    c1 = tf.keras.layers.Conv2D(16, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c1)
    p1 = tf.keras.layers.MaxPooling2D((2, 2))(c1)

    c2 = tf.keras.layers.Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(p1)
    c2 = tf.keras.layers.Dropout(0.1)(c2)
    c2 = tf.keras.layers.Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c2)
    p2 = tf.keras.layers.MaxPooling2D((2, 2))(c2)

    c3 = tf.keras.layers.Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(p2)
    c3 = tf.keras.layers.Dropout(0.2)(c3)
    c3 = tf.keras.layers.Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c3)
    p3 = tf.keras.layers.MaxPooling2D((2, 2))(c3)

    c4 = tf.keras.layers.Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(p3)
    c4 = tf.keras.layers.Dropout(0.2)(c4)
    c4 = tf.keras.layers.Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c4)
    p4 = tf.keras.layers.MaxPooling2D(pool_size=(2, 2))(c4)

    c5 = tf.keras.layers.Conv2D(256, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(p4)
    c5 = tf.keras.layers.Dropout(0.3)(c5)
    c5 = tf.keras.layers.Conv2D(256, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c5)

    #Expansive path 
    u6 = tf.keras.layers.Conv2DTranspose(128, (2, 2), strides=(2, 2), padding='same')(c5)
    u6 = tf.keras.layers.concatenate([u6, c4])
    c6 = tf.keras.layers.Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(u6)
    c6 = tf.keras.layers.Dropout(0.2)(c6)
    c6 = tf.keras.layers.Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c6)

    u7 = tf.keras.layers.Conv2DTranspose(64, (2, 2), strides=(2, 2), padding='same')(c6)
    u7 = tf.keras.layers.concatenate([u7, c3])
    c7 = tf.keras.layers.Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(u7)
    c7 = tf.keras.layers.Dropout(0.2)(c7)
    c7 = tf.keras.layers.Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c7)

    u8 = tf.keras.layers.Conv2DTranspose(32, (2, 2), strides=(2, 2), padding='same')(c7)
    u8 = tf.keras.layers.concatenate([u8, c2])
    c8 = tf.keras.layers.Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(u8)
    c8 = tf.keras.layers.Dropout(0.1)(c8)
    c8 = tf.keras.layers.Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c8)

    u9 = tf.keras.layers.Conv2DTranspose(16, (2, 2), strides=(2, 2), padding='same')(c8)
    u9 = tf.keras.layers.concatenate([u9, c1], axis=3)
    c9 = tf.keras.layers.Conv2D(16, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(u9)
    c9 = tf.keras.layers.Dropout(0.1)(c9)
    c9 = tf.keras.layers.Conv2D(16, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c9)

    outputs = tf.keras.layers.Conv2D(1, (1, 1), activation='sigmoid')(c9)

    model = tf.keras.Model(inputs=[inputs], outputs=[outputs])
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    return model

In [6]:
"""
model = create_model(IMG_HEIGHT, IMG_WIDTH, IMG_CHANNELS)
model.summary()

configproto = tf.compat.v1.ConfigProto() 
configproto.gpu_options.allow_growth = True
sess = tf.compat.v1.Session(config=configproto) 
tf.compat.v1.keras.backend.set_session(sess)
"""

'\nmodel = create_model(IMG_HEIGHT, IMG_WIDTH, IMG_CHANNELS)\nmodel.summary()\n\nconfigproto = tf.compat.v1.ConfigProto() \nconfigproto.gpu_options.allow_growth = True\nsess = tf.compat.v1.Session(config=configproto) \ntf.compat.v1.keras.backend.set_session(sess)\n'

In [7]:
def load_K14():
    model = create_model(IMG_HEIGHT, IMG_WIDTH, IMG_CHANNELS)
    #model.summary()
    model = tf.keras.models.load_model(checkpoint_dir)

    configproto = tf.compat.v1.ConfigProto() 
    configproto.gpu_options.allow_growth = True
    sess = tf.compat.v1.Session(config=configproto) 
    tf.compat.v1.keras.backend.set_session(sess)
    return model

In [8]:
def Train_K14(model):
    ################################
    #Modelcheckpoint
    checkpointer = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_dir, verbose=1, save_best_only=True)

    callbacks = [
            tf.keras.callbacks.EarlyStopping(patience=2, monitor='val_loss'),
            tf.keras.callbacks.TensorBoard(log_dir='logs')]

    results = model.fit(X_train, Y_train, validation_split=0.1, batch_size=16, epochs=1, callbacks=[checkpointer])
    ####################################

In [9]:
model = load_K14()

In [10]:
Train_K14(model)

Train on 2151 samples, validate on 240 samples
2144/2151 [============================>.] - ETA: 0s - loss: 0.0036 - accuracy: 0.9985
Epoch 00001: val_loss improved from inf to 0.15671, saving model to C:/Users/8holz/Dokumente/GitHub/K14/Training_Protocol/Model-Color-UNET-Terrosristhunt/weights
Instructions for updating:
If using Keras pass *_constraint arguments to layers.
INFO:tensorflow:Assets written to: C:/Users/8holz/Dokumente/GitHub/K14/Training_Protocol/Model-Color-UNET-Terrosristhunt/weights\assets
2151/2151 [==============================] - 66s 30ms/sample - loss: 0.0036 - accuracy: 0.9985 - val_loss: 0.1567 - val_accuracy: 0.9833


In [ ]:
for i in range(30):
    try:
        print("Epoch: "+str(i)+" is starting")
        model = load_K14()
        Train_K14(model)
        time.sleep(5)
        print("Epoch finished\n")
    except:
        print("Caught it!")
print("Training done so far\n")

Epoch: 0 is starting
Train on 2151 samples, validate on 240 samples
2144/2151 [============================>.] - ETA: 0s - loss: 0.0036 - accuracy: 0.9985
Epoch 00001: val_loss improved from inf to 0.15217, saving model to C:/Users/8holz/Dokumente/GitHub/K14/Training_Protocol/Model-Color-UNET-Terrosristhunt/weights
INFO:tensorflow:Assets written to: C:/Users/8holz/Dokumente/GitHub/K14/Training_Protocol/Model-Color-UNET-Terrosristhunt/weights\assets
2151/2151 [==============================] - 52s 24ms/sample - loss: 0.0036 - accuracy: 0.9985 - val_loss: 0.1522 - val_accuracy: 0.9836
Epoch finished

Epoch: 1 is starting
Train on 2151 samples, validate on 240 samples
2144/2151 [============================>.] - ETA: 0s - loss: 0.0035 - accuracy: 0.9985
Epoch 00001: val_loss improved from inf to 0.16462, saving model to C:/Users/8holz/Dokumente/GitHub/K14/Training_Protocol/Model-Color-UNET-Terrosristhunt/weights
INFO:tensorflow:Assets written to: C:/Users/8holz/Dokumente/GitHub/K14/Traini

INFO:tensorflow:Assets written to: C:/Users/8holz/Dokumente/GitHub/K14/Training_Protocol/Model-Color-UNET-Terrosristhunt/weights\assets
2151/2151 [==============================] - 54s 25ms/sample - loss: 0.0036 - accuracy: 0.9985 - val_loss: 0.1841 - val_accuracy: 0.9821
Epoch finished

Epoch: 14 is starting
Train on 2151 samples, validate on 240 samples
2144/2151 [============================>.] - ETA: 0s - loss: 0.0035 - accuracy: 0.9985
Epoch 00001: val_loss improved from inf to 0.17226, saving model to C:/Users/8holz/Dokumente/GitHub/K14/Training_Protocol/Model-Color-UNET-Terrosristhunt/weights
INFO:tensorflow:Assets written to: C:/Users/8holz/Dokumente/GitHub/K14/Training_Protocol/Model-Color-UNET-Terrosristhunt/weights\assets
2151/2151 [==============================] - 118s 55ms/sample - loss: 0.0035 - accuracy: 0.9985 - val_loss: 0.1723 - val_accuracy: 0.9832
Epoch finished

Epoch: 15 is starting
Train on 2151 samples, validate on 240 samples
2144/2151 [=======================

In [ ]:
#ls {checkpoint_dir}

In [ ]:
"""
idx = random.randint(0, len(X_train))


preds_train = model.predict(X_train[:int(X_train.shape[0]*0.9)], verbose=1)
preds_val = model.predict(X_train[int(X_train.shape[0]*0.9):], verbose=1)
preds_test = model.predict(X_test, verbose=1)

 
preds_train_t = (preds_train > 0.5).astype(np.uint8)
preds_val_t = (preds_val > 0.5).astype(np.uint8)
preds_test_t = (preds_test > 0.5).astype(np.uint8)
"""

"""
# Perform a sanity check on some random training samples
ix = random.randint(0, len(preds_train_t))
plt.imshow(X_train[ix])
plt.show()
plt.imshow(np.squeeze(Y_train[ix]))
plt.show()
plt.imshow(np.squeeze(preds_train_t[ix]))
plt.show()
"""
"""
def ImCompareGray(Im1, Im2, FigSize=(10,10)):
    f = plt.figure(figsize=FigSize)
    f.add_subplot(1,2, 1)
    plt.imshow(Im1)
    f.add_subplot(1,2, 2)
    plt.imshow(Im2)
    plt.show(block=True)

for i in range(50):
	ImCompareGray(X_train[i], np.squeeze(preds_train_t[i]))
"""

In [ ]:
"""
Dir = r"C:\Users\8holz\Dokumente\GitHub\K14\Training_Protocol\Model-Grayscale-UNET\images.pickle"
a = []
for i in X_train, Y_train, preds_train_t:
    a.append(i)
pickle.dump(a, open(Dir, "wb"))
"""

In [ ]:
"""
def Protocol(Dir, ModelName, Model, Img):
    #Model needs full path
    os.chdir(Dir)
    os.mkdir(ModelName)
    os.chdir(ModelName)
    os.system(" cp "+str(Model)+" "+str(Dir+ModelName+"Model.ckpt"))
"""